# Environments for Nested Definitions
Recall last time we defined the `make_adder` function, which
1. Takes in a formal parameter `n`
2. Creates another function within, `adder(k)`, which returns `k + n`
3. Returns the `adder` function, which was defined within its (`make_adder`) body

In [10]:
def make_adder(n):
    def adder(k):
        return k + n
    return adder

If we load this function, we can create a function `add_three` by calling `make_adder` on `3`,

In [11]:
add_three = make_adder(3)
add_three

<function __main__.make_adder.<locals>.adder(k)>

And now we have `add_three`, a function that adds 3 to things.

In [12]:
add_three(4)

7

In [13]:
add_three(7)

10

We see that part of the function `add_three` is the number `3` that gets added in. How do we have a function that has data in it? Let's analyze using environment diagram!

In [14]:
%%tutor --lang python3

def make_adder(n):
    def adder(k):
        return k + n
    return adder

add_three = make_adder(3)
result = add_three(4)

* In step 1, Python binds the function `make_adder` to the name `make_adder`
* In step 2, Python calls the function `make_adder` to `3`
* In step 3-5, Python creates a new environment named `make_adder`, where the formal parameter `n` is bound to `3`. 
    * Inside `make_adder`, Python defines the new function `adder`. 
    * Notice that the name `adder` is now bound to the function `adder(k)`. 
    * However, `adder` is currently only available in the `make_adder` frame

* In step 6, the return value of calling `make_adder` on `3` is the `adder` function! 
    * Thus, we can see in step 7 that the name `add_three` is bound to the functon `adder(k)`
* In step 8, Python tries to execute the `result = add_three(4)` line, which involves calling `add_three` on `4`. 
    * Thus, Python introduces a new frame `adder` with the formal parameter `k` bound to `4`.
    * Notice that the parent frame of `adder` is `f1`, which is the `make_adder` frame.
        * This way, Python still has access to the stuff within the `make_adder` frame
        
* In step 9, Python tries to compute the body of the `adder` function, `return k + n`. 
    * `k` is bound to `4`, in the `adder` frame, while `n` is bound to `3`, in the `make_adder` frame
    * Thus, the return value is
    
$$ 3 + 4 = 7 $$

## Environment Diagrams for Nested Def Statements
We'll analyze the environment diagram of the example above!

In the code, we have a nested `def` statements:
<img src = 'nested.jpg' width = 500/>

And we also have something new: a parent of a function that's not `Global` frame.

<img src = 'parent.jpg' width = 500/>

The `adder` function is defined in the body of the `make_adder` function. Thus, the `parent` frame of the `adder` function is the `make_adder` frame.

<img src = 'parent_2.jpg' width = 600/>

When we call the `adder` function, Python:
1. Copies the name `adder` to the new frame, 
2. Copies the formal parameter `k` and binds it to the argument value `4`
3. And copies the **parent of the function** as the **parent of the frame**

Why do we have `parent`s for functions? 

<img src = 'parent_function.jpg' width = 500/>

So that when we call those functions, Python will write down the correct `parent` for the frame,

<img src = 'correct_parent.jpg' width = 400/>

Why do we need `parent`s for frames? This tells us where to find the current environment.

<img src = 'parent_frame.jpg' width = 700/>

When Python evaluates the body of the `adder` function, `return k + n`, the current environment starts with the `adder` frame (labeled #1), followed by its parent, the `make_adder` frame (labeled #2), followed by its parent, the `Global` frame (labeled #3).

When Python looks up the value `k` in the environment, Python looks at the first frame of the current environemnt,  `adder`. It finds that `k` is `4`!

When Python looks up `n`, once again Python looks at the `adder` frame first. However, Python can't find `n` in `adder`! Thus, Python will look at the next frame, which is the `make_adder` frame. In this frame Python finds that `n` is bound to `3`

### Key Points
1. Every user-defined functions has a parent frame
    * In most examples we've gone so far, the parent frame is usually the `Global` frame
    * But in nested `def` statements, the inner `def`'s parent frame will be the outer `def`
    
2. The parent of a function is the frame in which it was defined

3. Every local frame has a parent frame

4. The parent of a frame is the parent of the function called

## How to Draw an Environment Diagram
Here is a guide on how to draw an environment diagram. This is useful so that we can draw one on our own without relying on PythonTutor. This is useful for understanding complicated examples.

### When a function is defined
Python always **create a function value** that looks like the following,

In [15]:
func <name>(<formal parameters>) [parent=<parent>]

SyntaxError: invalid syntax (<ipython-input-15-55b730d10fb9>, line 1)

Its parent is the current frame,

<img src = 'how_to.jpg' width = 500/>

When we created the `adder` function, Python execution was in `make_adder` frame. Thus, Python writes that `parent = f1`, in which `f1` the label for the frame `make_adder`. Python writes `parent = f1` instead of `parent = make_adder` since it's possible to have multiple frames named `make_adder`. By having a unique label (e.g. `f1`, `f2`), we can distinguish between each frame. 

Python binds `<name>` to the function value in the current frame. 

### When a function is called
1. Create (or add) a local frame, titled with the `<name>` of the function being called
2. Copy the parent of the function to the local frame:

In [ ]:
[parent=<label>]

3. Bind the `<formal parameters>` to the arguments in the local frame
4. Execute the body of the function in the environment that starts with the local frame
    * If Python needs to look up names in the environment, Python follows the parent of the parent of the parent...until it reaches the `Global` frame. Whichever frame Python found the name first, that's the value that Python uses.